In [2]:
%pwd

'c:\\Users\\Nandhakumar\\Documents\\End to End Projects\\solar-radiation-prediction\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Nandhakumar\\Documents\\End to End Projects\\solar-radiation-prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir : Path
    train_data_path: str
    test_data_path: str
    validation_data_path: str
    transformed_data_path: Path

In [6]:
from solarRadiation.constants import *
from solarRadiation.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir= config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path= config.test_data_path,
            validation_data_path=config.validation_data_path,
            transformed_data_path = config.transformed_data_path
        )

        return data_transformation_config

In [38]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

class DataTransformation:
    def __init__(self, config = DataTransformationConfig):
        self.config = config
        self.transformer = StandardScaler()
        self.target = "Clearsky DNI"

    def transform_data(self):
        train = pd.read_csv(self.config.train_data_path)
        test = pd.read_csv(self.config.test_data_path)
        validation = pd.read_csv(self.config.validation_data_path)

        tr_data = train.drop(["Clearsky DNI","Timestamp"], axis=1)

        col_list = list(tr_data.columns)

        high_correlation = []
        low_correlation = []
        for col in col_list:
            if (train[col].corr(train[self.target]) > 0.50) or (train[col].corr(train[self.target]) < -0.50):
                high_correlation.append(col)
            else:
                low_correlation.append(col)

        x_train = train.loc[:,high_correlation]
        x_test = test.loc[:, high_correlation]
        x_val = validation.loc[:, high_correlation]
        y_train = train[self.target]
        y_val = validation[self.target]

        X_train = self.transformer.fit_transform(x_train)
        X_test = self.transformer.transform(x_test)
        X_val = self.transformer.transform(x_val)

        create_directories([self.config.transformed_data_path])

        X_train.dump(os.path.join(self.config.transformed_data_path,'x_train.pkl'))
        X_test.dump(os.path.join(self.config.transformed_data_path,'x_test.pkl'))
        X_val.dump(os.path.join(self.config.transformed_data_path,'x_validation.pkl'))
        y_train.to_pickle(os.path.join(self.config.transformed_data_path,'y_train.pkl'))
        y_val.to_pickle(os.path.join(self.config.transformed_data_path,'y_validation.pkl'))


In [39]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.transform_data()
except Exception as e:
    raise e

[2023-11-28 09:24:53,548: INFO: common: yaml file config\config.yaml loaded successfully]
[2023-11-28 09:24:53,548: INFO: common: yaml file params.yaml loaded successfully]
[2023-11-28 09:24:53,548: INFO: common: Created directory at artifacts]
[2023-11-28 09:24:53,556: INFO: common: Created directory at artifacts/data_transformation]
[2023-11-28 09:24:55,601: INFO: common: Created directory at artifacts/data_transformation/data]
